In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv('2008_2021_data_matches.csv')
balls   = pd.read_csv('2008_2021_updated_ball.csv')

In [6]:
def get_run(ball, batsman = None, flag = 'Runs', innings = None, runs = None, choice = None, limit = 3, asc = False ):
    
    player                = ball[ball['batsman'].isin(batsman) ] if batsman else ball   
    temp_runs             = player.groupby(['batsman']).agg({'batsman_runs': 'sum'})
    
    runs_match            = player.groupby(['id']).agg({'batsman_runs':'sum'}).sort_values('batsman_runs',ascending = False)
    if len(temp_runs) == 0:
        tt = pd.DataFrame(columns = [ 'batsman','Innings','Not Out','Runs','Dots','HS','Avg','BF','SR','Hundered','Fifty','Four','Six' ] )
        tt = tt[choice] if choice else tt[['Innings','Not Out','Runs','Dots','HS','Avg','BF','SR','Hundered','Fifty','Four','Six']]
        tt['batsman'] = batsman
        return tt.fillna(0) 
    temp_runs['HS']       = player.groupby(['id','batsman']).agg({'batsman_runs':'sum'}).reset_index().groupby(['batsman']).agg({'batsman_runs':'max'})
    
    temp_runs_match       = player.groupby(['id','batsman']).agg({'batsman_runs':'sum'}).reset_index()
    temp_runs['Hundered'] = temp_runs_match[temp_runs_match['batsman_runs'] >= 100 ].groupby(['batsman']).agg({'batsman_runs':'count'})
    temp_runs['Fifty']    = temp_runs_match[ (temp_runs_match['batsman_runs'] < 100) & (temp_runs_match['batsman_runs'] >= 50) ].groupby(['batsman']).agg({'batsman_runs':'count'})
    temp_runs['Innings']  = temp_runs_match.groupby(['batsman']).agg({'id':'count'})
    
    temp_runs['No']       = ball[ball['player_dismissed'].isin(batsman)].groupby(['player_dismissed']).agg({'id':'count'}) if batsman else ball.groupby(['player_dismissed']).agg({'id':'count'})
    
    print(ball[ball['player_dismissed'].isin(batsman)].groupby(['player_dismissed']).agg({'id':'count'}))
    
    temp_runs['Not Out']  = temp_runs['Innings'] - temp_runs['No']
    temp_runs['Six']      = player[ (player['batsman_runs'] == 6) & (player['non_boundary'] == 0) ].groupby(['batsman']).agg({'batsman_runs':'count'})
    temp_runs['Four']     = player[ (player['batsman_runs'] == 4) & (player['non_boundary'] == 0) ].groupby(['batsman']).agg({'batsman_runs':'count'})
    temp_runs['Dots']     = player[player['batsman_runs'] == 0 ].groupby(['batsman']).agg({'batsman_runs':'count'})
    temp_runs['BF']       = player[~ player['extras_type'].isin(['wides'])].groupby(['batsman']).agg({'id':'count'})
    temp_runs['Avg']      = round(temp_runs['batsman_runs'] /(temp_runs['No'] ),2)
    temp_runs['SR']       = round((temp_runs['batsman_runs']/temp_runs['BF'])*100,2)
    temp_runs             = temp_runs.rename(columns = {'batsman_runs':'Runs'})

    temp_runs             = temp_runs.fillna(0) 
    temp_runs             = temp_runs.astype({'Six':int,'Four':int,'Hundered':int,'Fifty':int})
    temp_runs             = temp_runs[temp_runs['Innings'] >= innings] if innings else temp_runs    
    temp_runs             = temp_runs[temp_runs['Runs'] >= runs] if runs else temp_runs
    temp_runs             = temp_runs[choice] if choice else temp_runs[['Innings','Not Out','Runs','Dots','HS','Avg','BF','SR','Hundered','Fifty','Four','Six']]
    
    
    return temp_runs.sort_values(flag, ascending = asc ).head(limit).reset_index()
    


In [7]:
get_run(balls, ['S Dhawan'])

                   id
player_dismissed     
S Dhawan          167


,batsman,Innings,Not Out,Runs,Dots,HS,Avg,BF,SR,Hundered,Fifty,Four,Six
0,S Dhawan,191,24,5784,1797,106,34.63,4567,126.65,2,44,654,124


In [3]:
temp1 = balls.groupby(['id','batsman']).agg({'batsman_runs':'sum'})
temp2 = balls.groupby(['id','inning']).agg({'total_runs':'sum'})

In [4]:
dt = pd.merge(temp1, temp2, how = 'inner', left_on = 'id', right_on = 'id')

In [5]:
fin = dt.groupby(['id']).agg({'total_runs':'max','batsman_runs':'max'})

In [6]:
fin = fin[fin['total_runs'] >= 200 ]

In [7]:
fin.sort_values(['batsman_runs']).head(10)

,total_runs,batsman_runs
id,,
829763,200,57
829737,209,59
1136573,200,59
1082591,207,62
1136601,210,62
419114,218,63
734011,201,64
1237177,200,65
1082635,212,66


In [17]:
matches['team1'].unique()

array(['Delhi Daredevils', 'Chennai Super Kings',
       'Royal Challengers Bangalore', 'Mumbai Indians', 'Deccan Chargers',
       'Kings XI Punjab', 'Rajasthan Royals', 'Kolkata Knight Riders',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings'],
      dtype=object)

In [21]:
t1  = 'Chennai Super Kings'
t2  = 'Kolkata Knight Riders'

met = matches[ ((matches['team1'] == t1) & (matches['team2'] == t2) | (matches['team1'] == t2) & (matches['team2'] == t1) ) ]

In [ ]:
Year	Tournament	Venue	Winner	Margin
15-Oct-2021	2021 IPL	Dubai	CSK	27 Runs
26-Sep-2021	2021 IPL	Abu Dhabi	CSK	2 Wickets
21-Apr-2021	2021 IPL	Mumbai	CSK	18 Runs
29-Oct-2020	2020 IPL	Dubai	CSK	6 Wickets
7-Oct-2020	2020 IPL	Abu Dhabi	KKR	10 Runs
14-Apr-2019	2019 IPL	Kolkata	CSK	5 Wickets
9-Apr-2019	2019 IPL	Chennai	CSK	7 Wickets
3-May-2018	2018 IPL	Kolkata	KKR	6 wickets
10-Apr-2018	2018 IPL	Chennai	CSK	5 wickets
29-Apr-2015	2015 IPL	Kolkata	KKR	7 wickets
27-Apr-2015	2015 IPL	Chennai	CSK	2 runs
3-Oct-2014	2014 CLT20	Bangalore*	CSK	8 wickets
16-Sep-2014	2014 CLT20	Hyderabad*	KKR	3 wickets
19-May-2014	2014 IPL	Ranchi	CSK	34 runs
1-May-2014	2014 IPL	Kolkata	KKR	8 wickets
27-Apr-2013	2013 IPL	Kolkata	CSK	4 wickets
19-Apr-2013	2013 IPL	Chennai	CSK	14 runs
26-May-2012	2012 IPL	Chennai	KKR	5 wickets
13-May-2012	2012 IPL	Kolkata	CSK	5 wickets
29-Apr-2012	2012 IPL	Chennai	KKR	5 wickets
6-May-2011	2011 IPL	Chennai	CSK	2 runs
7-Apr-2011	2011 IPL	Kolkata	KKR	10 runs
12-Apr-2010	2010 IPL	Kolkata	CSK	55 runs
15-Mar-2010	2010 IPL	Chennai	CSK	9 wickets
17-May-2009	2009 IPL	Cape Town	-	-
24-Apr-2009	2009 IPL	Centurion	KKR	7 wickets
17-May-2008	2008 IPL	Chennai	CSK	9 wickets
25-Apr-2008	2008 IPL	Kolkata	CSK	3 runs

In [22]:
met[['date','venue','winner','result','result_margin']]

,date,venue,winner,result,result_margin
5,26-04-2008,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,wickets,9.0
30,18-05-2008,Eden Gardens,Chennai Super Kings,runs,3.0
70,18-05-2009,SuperSport Park,Kolkata Knight Riders,wickets,7.0
141,16-03-2010,Eden Gardens,Chennai Super Kings,runs,55.0
153,13-04-2010,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,wickets,9.0
201,08-04-2011,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,runs,2.0
215,07-05-2011,Eden Gardens,Kolkata Knight Riders,runs,10.0
255,14-05-2012,Eden Gardens,Chennai Super Kings,wickets,5.0
272,27-05-2012,"MA Chidambaram Stadium, Chepauk",Kolkata Knight Riders,wickets,5.0
315,30-04-2012,"MA Chidambaram Stadium, Chepauk",Kolkata Knight Riders,wickets,5.0
